## Paralelismo com OpenMP

Até agora foram implementadas três estratégias sequenciais para o problema de alinhamento de sequencias de DNA: busca heurística, busca local e busca exaustiva. Neste projeto foi realizado a paralelização da busca exaustiva com o alinhamento por meio de Smith-Waterman.

Para isso, a estratégia para paralelizar foi encontrar a area de maior demanda de código por meio das ferramentas de profiling e, a partir disso, foi possível identificar que era na chamada da função de calculo dos scores. Com essa informação em mente foi possível aplicar o openmp nesse ponto.

### Execução

Inicialmente, foi utilizado o programa apresentado abaixo para gerar a entrada que o algoritmo utilizará. Este gera um arquivo txt apelidado de **dna.seq** com quatro linhas, as quais indicam, respectivamente:
- o tamanho da primeira sequência
- o tamanho da segunda sequência
- a primeira sequência 
- a segunda sequência

In [1]:
import random
n = 10 # tamanho da primeira sequência
m = 40 # tamanho da segunda sequência
file = 'dna.seq' # nome do arquivo a ser gerado
f = open(file, 'w')
seq=[str(n)+'\n',
     str(m)+'\n',
     ''.join(random.choices(['A','T','C','G','-'],k=n))+'\n',
     ''.join(random.choices(['A','T','C','G','-'],k=m))]
f.writelines(seq)
f.close()
print(''.join(seq))

10
40
-GCTC-TCCC
ATG-TACCAATTTAACCCGGT-GTTA-TAAATCCA-AAC-


Assim, com o arquivo de entrada estabelecido, o próximo passo realizado no projeto foi compilar o mesmo com o comando apresentado abaixo, gerando um executável denominado **main**.

`g++ -Wall -O3 main.cpp -o main`

Com o executável gerado, o código abaixo permite que o programa desenvolvido em c++ seja rodado, recebendo como entradas o arquivo dna.seq, criado acima, e gerando uma resposta com as sequências alinhadas e em quanto tempo este programa rodou.

In [4]:
import subprocess
import time

with open('dna.seq') as f:
    start = time.perf_counter()
    proc = subprocess.run(['./main'], input=f.read(), text=True, capture_output=True)
    end = time.perf_counter()

    print('Saída:', proc.stdout)
    print('Tempo total(s):', end - start)

Saída: Score maximo: 8
Primeira sequencia utilizada: TC-TCCC
Segunda sequencia utilizada: TG-TACCAATTTAACCCGGT-GTTA-TAAATCCA-AAC-

Tempo total(s): 0.03173673298442736


Para fins de comparação, foi rodado a implementação da busca exaustiva por smith-waterman sem a paralelização para ver em quanto tempo o programa rodaria.

In [5]:
import subprocess
import time

with open('dna.seq') as f:
    start = time.perf_counter()
    proc = subprocess.run(['./main_sw'], input=f.read(), text=True, capture_output=True)
    end = time.perf_counter()

    print('Saída:', proc.stdout)
    print('Tempo total(s):', end - start)

Saída: Score maximo: 8
Primeira sequencia utilizada: -GCTC-T
Segunda sequencia utilizada: ATG-TACCAATTTAACCCGGT-GTTA-T

Tempo total(s): 0.05143294500885531


A partir disso, foi possível perceber que a paralelização realmente otimiza o código em questão de eficiência de tempo, percebendo uma diferença de 0.02s.

------------------------------
## Validação

Para validar se os resultados obtidos pelo programa eram o esperado, foi utilizado o [Simulador online do Algoritmo de Smith-Waterman](http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Smith-Waterman). As imagens apresentadas abaixo indicam que os resultados foram de fato iguais.

Sequências utilizadas:

```
-GCTC-TCCC
ATG-TACCAATTTAACCCGGT-GTTA-TAAATCCA-AAC-
```

- Resultado do algoritmo implementado

``` 
Saída: Score maximo: 8
Primeira sequencia utilizada: TC-TCCC
Segunda sequencia utilizada: TG-TACCAATTTAACCCGGT-GTTA-TAAATCCA-AAC-
``` 

- Resultado do Simulador


![input](img/simulador.jpg)